In [66]:
# Import libraries

import sqlalchemy
import pandas as pd 
from sqlalchemy.orm import sessionmaker
import json
from datetime import datetime
import datetime
import sqlite3
from sqlite3 import Error
from bs4 import BeautifulSoup

# Extract

In [56]:
with open('database.txt') as f:
    json_data = json.load(f)    

# Transform/ Validate

In [3]:
# Check if the DataFrame is empty

In [4]:
# Primary key check (steam_app_id)

In [5]:
# Check for NULL values

# Load

In [6]:
# save to a database

In [7]:
database = r"C:\Users\kasia\pyproj\deployment\steam_data.sqlite"

In [26]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.executescript(create_table_sql)
    except Error as e:
        print(e)


def main():
    #database = r"C:\Users\kasia\pyproj\deployment\steam_data.sqlite"

    create_table_games = """
        DROP TABLE games;
        CREATE TABLE games(
            steam_appid INTEGER,
            game_type TEXT(50),
            game_name TEXT(250),
            required_age INTEGER,
            is_free INTEGER,
            detailed_description TEXT(400),
            about_the_game TEXT(400),
            short_description TEXT(100),
            languages TEXT(350),
            num_reviews INTEGER,
            header_image TEXT(250),
            website TEXT,
            pc_requirements_min TEXT,
            pc_requirements_recomm TEXT,
            mac_requirements_min TEXT,
            mac_requirements_recomm TEXT,
            legal_notice TEXT,
            price_currency TEXT(15),
            price_initial INTEGER,
            price_final INTEGER,
            price_discount_percent INTEGER,
            price_initial_formatted TEXT(15),
            price_final_formatted TEXT(15),
            release_date_coming_soon INTEGER,
            release_date TEXT(15),
            support_info_url TEXT(100),
            support_info_email TEXT(60),
            support_info_background TEXT(250),
            CONSTRAINT games_PK PRIMARY KEY (steam_appid)
        );

        """
    create_table_developers = """  
    DROP TABLE developers;
    CREATE TABLE developers (
        developer_id INTEGER PRIMARY KEY AUTOINCREMENT,
        developer TEXT(100)
    );
        """
    create_table_game_dev = """  
    DROP TABLE game_dev;
    CREATE TABLE game_dev (
        steamapp_id INTEGER,
        developer_id INTEGER,
        CONSTRAINT game_dev_FK FOREIGN KEY (developer_id) REFERENCES developers(developer_id),
        CONSTRAINT game_dev_FK_1 FOREIGN KEY (steamapp_id) REFERENCES games(steam_appid)
    );
    """ 
    create_table_publishers = """  
    DROP TABLE publishers;
    CREATE TABLE publishers (
        publisher_id INTEGER PRIMARY KEY AUTOINCREMENT,
        publisher TEXT
    );   
    """ 
    create_table_game_publ = """   
    DROP TABLE game_publ; 
    CREATE TABLE game_publ (
        publisher_id INTEGER,
        gameapp_id INTEGER,
        CONSTRAINT game_publ_FK FOREIGN KEY (gameapp_id) REFERENCES games(steam_appid),
        CONSTRAINT game_publ_FK_1 FOREIGN KEY (publisher_id) REFERENCES publishers(publisher_id)
    );    
    """ 
    create_table_platforms = """  
    DROP TABLE platforms;   
    CREATE TABLE platforms (
        platform_id INTEGER PRIMARY KEY AUTOINCREMENT,
        platform TEXT(15)
    );    
    """ 
    create_table_game_platform = """   
    DROP TABLE game_platform;  
    CREATE TABLE game_platform (
        gameapp_id INTEGER,
        platform_id INTEGER,
        CONSTRAINT game_platform_FK FOREIGN KEY (gameapp_id) REFERENCES games(steam_appid),
        CONSTRAINT game_platform_FK_1 FOREIGN KEY (platform_id) REFERENCES platforms(platform_id)
        );    
    """ 
    create_table_categories = """   
    DROP TABLE categories;  
    CREATE TABLE categories (
        category_id INTEGER,
        category TEXT(30),
        CONSTRAINT Categories_PK PRIMARY KEY (category_id)
    );   
    """ 
    create_table_game_category = """   
    DROP TABLE game_category;  
    CREATE TABLE game_category (
        gameapp_id INTEGER,
        category_id INTEGER,
        CONSTRAINT game_category_FK FOREIGN KEY (gameapp_id) REFERENCES games(steam_appid),
        CONSTRAINT game_category_FK_1 FOREIGN KEY (category_id) REFERENCES categories(category_id)
    );  
    """ 
    create_table_genres = """   
    DROP TABLE genres;   
    CREATE TABLE genres (
        genre_id INTEGER,
        genre TEXT(30),
        CONSTRAINT genres_PK PRIMARY KEY (genre_id)
    );  
    """ 
    create_table_game_genre = """   
    DROP TABLE game_genre;   
    CREATE TABLE game_genre (
        gameapp_id INTEGER,
        genre_id INTEGER,
        CONSTRAINT game_genre_FK FOREIGN KEY (gameapp_id) REFERENCES games(steam_appid),
        CONSTRAINT game_genre_FK_1 FOREIGN KEY (genre_id) REFERENCES genres(genre_id)
    );   
    """ 
    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        create_table(conn, create_table_games)
        create_table(conn, create_table_developers)
        create_table(conn, create_table_game_dev)
        create_table(conn, create_table_publishers)
        create_table(conn, create_table_game_publ)
        create_table(conn, create_table_platforms)
        create_table(conn, create_table_game_platform)
        create_table(conn, create_table_categories)
        create_table(conn, create_table_game_category)
        create_table(conn, create_table_genres)
        create_table(conn, create_table_game_genre)
    else:
        print("Error! cannot create the database connection.")


if __name__ == '__main__':
    main()

In [9]:
# Insert data to the SQL tables

In [17]:
def insert_game(conn, game):
    """
    Create a new game into the games table
    :param conn:
    :param game:
    :return: game id
    """
    sql = ''' INSERT INTO games(
        steam_appid,
        game_type,
        game_name,
        required_age,
        is_free,
        detailed_description,
        about_the_game,
        short_description,
        language,
        num_reviews,
        header_image,
        website,
        pc_requirements_min,
        mac_requirements_min,
        legal_notice,
        price_currency,
        price_initial,
        price_final,
        price_discount_percent,
        price_initial_formatted,
        price_final_formatted,
        release_date_coming_soon,
        release_date,
        support_info_url,
        support_info_email,
        support_info_background)
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, game)
    conn.commit()
    return cur.lastrowid

def insert_developer(conn, developer):
    """
    Create a new developer into the developers table
    :param conn:
    :param developer:
    :return: developer id
    """
    sql = ''' INSERT INTO developers (
        developer_id,
        developer)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, developer)
    conn.commit()
    return cur.lastrowid

def insert_game_dev(conn, game_dev):
    """
    Create a new game_dev into the game_dev table
    :param conn:
    :param game_dev:
    :return: developer id
    """
    sql = ''' INSERT INTO game_dev (
        steamapp_id,
        developer_id)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, game_dev)
    conn.commit()
    return cur.lastrowid

def insert_publisher(conn, publisher):
    """
    Create a new publisher into the publishers table
    :param conn:
    :param publisher:
    :return: publisher id
    """
    sql = ''' INSERT INTO publishers (
        publisher_id,
        publisher)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, publisher)
    conn.commit()
    return cur.lastrowid

def insert_game_publ(conn, game_publ):
    """
    Create a new game_publ into the game_publ table
    :param conn:
    :param game_publ:
    :return: publisher id
    """
    sql = ''' INSERT INTO game_publ (
        publisher_id,
        gameapp_id)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, game_publ)
    conn.commit()
    return cur.lastrowid

def insert_platform(conn, platform):
    """
    Create a new platform into the platforms table
    :param conn:
    :param platform:
    :return: publisher id
    """
    sql = ''' INSERT INTO platforms (
        platform_id,
        platform)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, platform)
    conn.commit()
    return cur.lastrowid

def insert_game_platform(conn, game_platform):
    """
    Create a new game_platform into the game_platform table
    :param conn:
    :param game_platform:
    :return: game_platform id
    """
    sql = ''' INSERT INTO game_platform (
        gameapp_id,
        platform_id)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, game_platform)
    conn.commit()
return cur.lastrowid

def insert_category(conn, category):
    """
    Create a new category into the categories table
    :param conn:
    :param category:
    :return: category id
    """
    sql = ''' INSERT INTO categories (
        category_id,
        category)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, category)
    conn.commit()
    return cur.lastrowid

def insert_game_category(conn, game_category):
    """
    Create a new game_category into the game_category table
    :param conn:
    :param game_category:
    :return: game_category id
    """
    sql = ''' INSERT INTO game_category (
        gameapp_id ,
        category_id )
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, game_category)
    conn.commit()
    return cur.lastrowid

def insert_genre(conn, genre):
    """
    Create a new genre into the genre table
    :param conn:
    :param genre:
    :return: genre id
    """
    sql = ''' INSERT INTO genres (
        genre_id,
        genre)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, genre)
    conn.commit()
    return cur.lastrowid

def insert_game_genre(conn, game_genre):
    """
    Create a new game_category into the game_genre__category table
    :param conn:
    :param game_genre:
    :return: genre id
    """
    sql = ''' INSERT INTO game_genre (
        gameapp_id,
        genre_id)
        VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, game_genre)
    conn.commit()
    return cur.lastrowid

def main():
    # create a database connection
    conn = create_connection(database)
    with conn:
        # create a new entry
        for key,game in json_data.items():
            steam_appid = game['steam_appid']
            game_type = game['type']
            game_name = game['name']
            required_age = game['required_age']
            is_free = game['is_free']
            detailed_description = game['detailed_description']
            about_the_game = game['about_the_game']
            short_description = game['short_description']
            language = game['supported_languages']
            num_reviews = game['num_reviews']
            header_image = game['header_image']
            website = game['website']
            pc_requirements_min = game['pc_requirements']['minimum']
            mac_requirements_min = game['mac_requirements']['minimum']
            legal_notice = game['legal_notice']
            price_currency = game['price_overview']['currency']
            price_initial = game['price_overview']['initial']
            price_final = game['price_overview']['final']
            price_discount_percent = game['price_overview']['discount_percent']
            price_initial_formatted = game['price_overview']['initial_formatted']
            price_final_formatted = game['price_overview']['final_formatted']
            release_date_coming_soon = game['release_date']['coming_soon']
            release_date = game['release_date']['date']
            support_info_url = game['support_info']['url']
            support_info_email = game['support_info']['email']
            support_info_background = game['background']


if __name__ == '__main__':
    main()

SyntaxError: 'return' outside function (Temp/ipykernel_21084/3161288988.py, line 165)

In [ ]:
# read data of one game

# visualize

# Create a page

In [ ]:
# create a page in flask

# Create a container

In [ ]:
# create a container in Docker

# Deploy

In [ ]:
# deploy with Heroku